In [12]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString
from busroute import snap_points_to_line, split_routes

# busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_seq_col = 'Seq'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

shp_outputfolder = os.path.join(os.getcwd(),'..', 'output','shp')
os.makedirs(shp_outputfolder, exist_ok=True)

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))

In [13]:
# 只做同時有站序 & 路線檔案的
routelist = list(set(list(busroute[route_routename_col])) & set(list(seq[seq_routename_col])))
print("可計算的路線共有:", len(routelist),'條')
only_in_route = list(set(busroute[route_routename_col]) - set(seq[seq_routename_col]))
print("只有路線檔案的路線:", only_in_route)
only_in_seq = list(set(seq[seq_routename_col]) - set(busroute[route_routename_col]))
print("只有站序檔案的路線:", only_in_seq)

可計算的路線共有: 379 條
只有路線檔案的路線: []
只有站序檔案的路線: []


In [ ]:
allgeodataframe = []
for route in routelist:
    for direction in [0,1]:
        try:
            # 取得對應的方向的route
            busroute_select = busroute[ (busroute[route_routename_col] == route) & (busroute[route_direction_col] == direction)][[route_routename_col,route_direction_col,'geometry' ]].reset_index(drop = True)
            # 也要有對應的seq
            seq_select = seq[ (seq[seq_routename_col] == route) & (seq[seq_direction_col] == direction) ].sort_values(seq_seq_col).reset_index(drop = True)
            seq_select['geometry'] = seq_select.apply(lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1)
            # 將seq_select從 Pandas DataFrame 轉換為 GeoDataFrame
            seq_select = gpd.GeoDataFrame(seq_select, geometry='geometry').drop_duplicates(subset=[seq_seq_col]).reset_index(drop = True)
            seq_select = seq_select.set_crs(epsg=4326, inplace=True)

            # (1) 把公車站點（投影到路線上）
            # 假設 busroute_select 和 seq_select 已經是 GeoDataFrames，並且你已經知道欄位名稱
            seq_select = snap_points_to_line(seq_select, busroute_select, 
                                                    route_routename_col=route_routename_col, 
                                                    route_direction_col=route_direction_col, 
                                                    seq_routename_col=seq_routename_col, 
                                                    seq_direction_col=seq_direction_col, 
                                                    seq_lat_col=seq_lat_col, 
                                                    seq_lng_col=seq_lng_col)
            
            # (2) 把路線進行拆分
            segment = split_routes(busroute_select, seq_select, route_routename_col = 'RouteNameZ').set_crs(epsg=4326, inplace=True)
            segment = segment.to_crs(epsg=3826) # 轉換為TWD97的才有辦法進行計算
            segment['length'] = segment.geometry.length # 計算長度，這樣長度將以米為單位
            segment['Direction'] = segment['Direction'].astype(int)
            segment['OD'] = segment['StartSeq'].astype(str) + "-" + segment['EndSeq'].astype(str)
            segment = segment[['RouteName', 'OD', 'Direction', 'StartSeq', 'EndSeq', 'length' ,'geometry']]
            allgeodataframe.append(segment)
        except:
            pass
allgeodataframe = pd.concat(allgeodataframe)
allgeodataframe = allgeodataframe.reset_index(drop = True)
# allgeodataframe.to_file(os.path.join(shp_outputfolder, 'busroute_segment.shp'))
# 篩選出只包含 LINESTRING 類型的資料
line_segments = allgeodataframe[allgeodataframe.geometry.type == 'LineString']
# 寫入 Shapefile
line_segments.to_file(os.path.join(shp_outputfolder, 'busroute_segment.shp'))

print("目前沒有拆分路段的BusRoute如下：")
print( list(set(routelist) - set (line_segments['RouteName']) ))